In [1]:
#import required libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_datasets as tfds

In [2]:
# Downloading the GloVe embeddings from Stanford's website
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d glove.6B


--2023-09-28 00:10:52--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-09-28 00:10:52--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-09-28 00:10:53--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [3]:
# 1. Load the AG News dataset
dataset, info = tfds.load('ag_news_subset', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

# Tokenization and sequence creation
tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")
train_texts = [x[0].numpy().decode('utf-8') for x in train_dataset]
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
train_sequences = pad_sequences(train_sequences, padding='post')
max_length = train_sequences.shape[1]

test_texts = [x[0].numpy().decode('utf-8') for x in test_dataset]
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_sequences = pad_sequences(test_sequences, padding='post', maxlen=max_length)

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/120000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/1.0.0.incomplete9NY81K/ag_news_subset-train.tfrecord*...:  …

Generating test examples...:   0%|          | 0/7600 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/1.0.0.incomplete9NY81K/ag_news_subset-test.tfrecord*...:   …

Dataset ag_news_subset downloaded and prepared to /root/tensorflow_datasets/ag_news_subset/1.0.0. Subsequent calls will reuse this data.


In [4]:
# 2. Use Pretrained Word Embeddings
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Download GloVe embeddings and prepare embedding matrix
with open('/content/glove.6B/glove.6B.50d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        if word in tokenizer.word_index:
            idx = tokenizer.word_index[word]
            embedding_matrix[idx] = np.array(values[1:], dtype=np.float32)

# 3. Build, Compile, and Fit the Model
model_lstm = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False),
    LSTM(32, return_sequences=True),
    LSTM(32),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')
])

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical([label.numpy() for _, label in train_dataset])
test_labels = tf.keras.utils.to_categorical([label.numpy() for _, label in test_dataset])

model_lstm.fit(train_sequences, train_labels, epochs=10, validation_split=0.2)

# 4. Evaluate the Model on the Test Set
loss, accuracy = model_lstm.evaluate(test_sequences, test_labels)
print("Loss:", loss)
print("Accuracy:", accuracy)


Epoch 1/10
3000/3000 [==============================] - 397s 131ms/step - loss: 1.3865 - accuracy: 0.2513 - val_loss: 1.3855 - val_accuracy: 0.2477
Epoch 2/10
3000/3000 [==============================] - 391s 130ms/step - loss: 0.7995 - accuracy: 0.6212 - val_loss: 0.4659 - val_accuracy: 0.8465
Epoch 3/10
3000/3000 [==============================] - 386s 129ms/step - loss: 0.4149 - accuracy: 0.8600 - val_loss: 0.3691 - val_accuracy: 0.8739
Epoch 4/10
3000/3000 [==============================] - 389s 130ms/step - loss: 0.3598 - accuracy: 0.8782 - val_loss: 0.3472 - val_accuracy: 0.8850
Epoch 5/10
3000/3000 [==============================] - 384s 128ms/step - loss: 0.3492 - accuracy: 0.8833 - val_loss: 0.3383 - val_accuracy: 0.8835
Epoch 6/10
3000/3000 [==============================] - 390s 130ms/step - loss: 0.3196 - accuracy: 0.8911 - val_loss: 0.3209 - val_accuracy: 0.8878
Epoch 7/10
3000/3000 [==============================] - 389s 130ms/step - loss: 0.3068 - accuracy: 0.8953 - val_